In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

import momepy as mm
import networkx as nx

import math

from contextlib import redirect_stdout

import io
import sys

from shapely.ops import nearest_points
from shapely.ops import unary_union

In [ ]:
# проверка остановки в буффере - если даже по вохдуху не попадает - не искать

In [2]:
graph = gpd.read_file('../lines_to_graph/with_ped/new_graph_with_ped.json',
                        encoding='utf-8')

In [3]:
nodes = gpd.read_file('../lines_to_graph/with_ped/all_nodes_with_ped.json',
                        encoding='utf-8')

In [4]:
buildings = gpd.read_file('./all_bui.json',encoding='utf-8')

In [5]:
stps_big = pd.read_csv('../lines_to_graph/total/df_stop_seq_all_n_samobl.csv',
                        encoding='utf-8', sep=';')

In [6]:
stps_big = stps_big[['stop_id', 'stop_name', 'lat', 'lon']]
stps_big = stps_big.drop_duplicates(subset=['stop_id']).reset_index(drop=True)

In [7]:
lst_geo=[]
lst_lat=list(stps_big.lat)
lst_lon=list(stps_big.lon)
i=0
for i in tqdm(range(len(stps_big))):
    one_pt = Point(lst_lon[i], lst_lat[i])
    lst_geo.append(one_pt)
# 

In [17]:
gdf_stops_all = gpd.GeoDataFrame(stps_big, geometry=lst_geo)
gdf_stops_all.crs='epsg:4326'

In [16]:
border_sa = gpd.read_file('./data/outer_border_SA.shp',encoding='utf-8')

In [18]:
gdf_stops = gpd.sjoin(gdf_stops_all, border_sa[['geometry']], how='inner', op='intersects')
gdf_stops = gdf_stops.drop('index_right', axis=1).reset_index(drop=True)

In [9]:
# убрать жд пути из графа
graph_ped = graph[graph.type_ped != 'no_ped'].reset_index(drop=True)[['link_id', 'geometry']]

In [10]:
shrt_bui = buildings[['osm_id', 'geometry']].copy()

In [11]:
def intersect_using_spatial_index(source_gdf, intersecting_gdf):
    """
    Conduct spatial intersection using spatial index for candidates GeoDataFrame to make queries faster.
    Note, with this function, you can have multiple Polygons in the 'intersecting_gdf' and it will return all the points
    intersect with ANY of those geometries.
    """
    source_sindex = source_gdf.sindex
    possible_matches_index = []

    # 'itertuples()' function is a faster version of 'iterrows()'
    for other in intersecting_gdf.itertuples():
        bounds = other.geometry.bounds
        c = list(source_sindex.intersection(bounds))
        possible_matches_index += c

    # Get unique candidates
    unique_candidate_matches = list(set(possible_matches_index))
    possible_matches = source_gdf.iloc[unique_candidate_matches]

    # Conduct the actual intersect
    result = possible_matches.loc[possible_matches.intersects(intersecting_gdf.unary_union)]
    return result
########

In [12]:
def find_closest_stop(gdf_one_bui, gdf_stops, gdf_buff):
    uu_stops = gdf_stops.geometry.unary_union
    centr_bui = gdf_one_bui.geometry[0].centroid
    nrst_pt = nearest_points(centr_bui, uu_stops)[1]
    
    nearest_stop_id = list(gdf_stops[gdf_stops.geometry == nrst_pt]['stop_id'])[0]
    nearest_stop_geo = list(gdf_stops[gdf_stops.geometry == nrst_pt]['geometry'])[0]
    if gdf_buff.geometry[0].intersects(nearest_stop_geo):
        return [nearest_stop_id, nearest_stop_geo]
    else:
        return []

In [13]:
def get_closest_node(gdf_obj, linkid, graph_in_buff):
    line_geo = list(graph_in_buff[graph_in_buff.link_id == linkid]['geometry'])[0]
    start = Point(line_geo.coords[0])
    end = Point(line_geo.coords[-1])
    obj_centr = gdf_obj['geometry'][0]#.centroid

    if start.distance(obj_centr) < end.distance(obj_centr):
        return start
    else:
        return end
# 

In [14]:
def get_shrt_pth(gdf_one_bui, gdf_buff, gdf_stops, graph_ped):

    try:
        nearest_stop = find_closest_stop(gdf_one_bui, gdf_stops, gdf_buff)
    except:
        nearest_stop = []
    #
    if nearest_stop != []:
        gdf_near_stop = gpd.GeoDataFrame(data=[nearest_stop], columns=['stop_id', 'geometry'])
        gdf_near_stop.crs=graph_ped.crs
        graph_in_buff = intersect_using_spatial_index(graph_ped, gdf_buff[['geometry']])
        centr_bui = gdf_one_bui.copy()
        centr_bui.geometry = centr_bui.geometry.centroid

        text_trap = io.StringIO()
        with redirect_stdout(text_trap):
            closest_link_bui = mm.get_network_id(gdf_one_bui, graph_in_buff, 'link_id')[0]
            closest_link_stop = mm.get_network_id(gdf_near_stop, graph_in_buff, 'link_id')[0]

        closest_node_bui = get_closest_node(gdf_one_bui, closest_link_bui, graph_in_buff)
        closest_node_stop = get_closest_node(gdf_near_stop, closest_link_stop, graph_in_buff)

        one_G = mm.gdf_to_nx(graph_in_buff)
        res_lst = nx.shortest_path(one_G, closest_node_bui.coords[0], closest_node_stop.coords[0])

        if len(res_lst) == 1:
            line_path = LineString([closest_node_bui, closest_node_stop])
            path_lngth = 0
        elif len(res_lst) == 0:
            line_path = LineString([closest_node_bui, closest_node_stop])
            path_lngth = -1
        else:
            line_path = LineString(res_lst)
            path_lngth = 1
    # 
        return [nearest_stop[0], path_lngth, res_lst, line_path]
    else:
        return []

In [58]:
def generate_graph(edges, fields):
    G = nx.Graph()

    key = 0
    for index, row in edges.iterrows():
        first = row.from_node
        last = row.to_node

        data = [row[f] for f in fields]
        attributes = dict(zip(fields, data))
        G.add_edge(first, last, key=key, **attributes)
        key += 1
    # 
    return G

In [ ]:
def get_lst_nodes(egg):
    lst_nodes_end=[]
    lst_nodes = list(egg.nodes())
    i=0
    for i in (range(len(lst_nodes))):
        one_node = lst_nodes[i]
        lst_neigh = list(nx.all_neighbors(egg, one_node))
        if len(lst_neigh) < 2:
            lst_nodes_end.append(one_node)
    #
    return lst_nodes_end

In [ ]:
def get_strt_end_paths(egg, closest_node, lst_nodes_end):
    
    shortest_paths = list(nx.all_pairs_dijkstra_path(egg))
    
    start_ind=0
    i=0
    for i in range(len(shortest_paths)):
        if shortest_paths[i][0] == closest_node:
            start_ind = i
            break
    # 
    lst_paths_to_ends = []
    i=0
    for nd in shortest_paths[start_ind][1]:
        if nd in lst_nodes_end:
            lst_paths_to_ends.append(shortest_paths[start_ind][1][nd])
    # 
    lst_paths_to_ends.remove([closest_node])
    return lst_paths_to_ends

In [15]:
# list(shrt_bui.osm_id).index(40707382)

In [15]:
# np_shbu = shrt_bui.to_numpy()
# ind_oi_bui = list(shrt_bui.columns).index('osm_id')
# ind_geo_bui = list(shrt_bui.columns).index('geometry')

In [16]:
# lst_total_path = []
# i=0
# # for i in tqdm(range(len(np_shbu))):
# for i in tqdm(range(2)):
#     one_bui_oi = np_shbu[i, ind_oi_bui]
#     one_bui_geo = np_shbu[i, ind_geo_bui]
    
#     gdf_one_bui = gpd.GeoDataFrame(geometry=[one_bui_geo])
#     gdf_one_bui.crs = 'epsg:4326'

#     gdf_buff = gdf_one_bui.copy()
#     gdf_buff.geometry = gdf_buff.geometry.to_crs('epsg:32637').buffer(1000).to_crs('epsg:4326')
    
#     lst_path = get_shrt_pth(gdf_one_bui, gdf_buff, gdf_stops, graph_ped)
    
#     if lst_path != []:
#         lst_total_path.append([one_bui_oi] + lst_path)
#     #
# # 

C:\Users\ochristm\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  del sys.path[0]
C:\Users\ochristm\anaconda3\lib\site-packages\momepy\elements.py:696: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings_c["geometry"] = buildings_c.centroid  # make centroids
Snapping: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.02it/s]
C:\Users\ochristm\anaconda3\lib\site-packages\momepy\utils.py:202: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_netwo

In [18]:
# gdf_path = gpd.GeoDataFrame(lst_total_path, 
#                             columns = ['osm_id_bui','stop_id_ya',
#                                        'is_valid', 'node_seq', 'geometry'])
# gdf_path.crs = graph_ped.crs

In [64]:
graph_notdir = graph[graph.direction == 'direct'].reset_index(drop=True)
graph_notdir = graph_notdir[['link_id', 'from_node', 'to_node', 'geometry']]

In [65]:
graph_notdir_32640 = graph_notdir.copy()
graph_notdir_32640 = graph_notdir_32640.to_crs('epsg:32640')
graph_notdir_32640['length'] = graph_notdir_32640.geometry.length
fields = list(graph_notdir_32640.columns)

In [66]:
# https://github.com/martinfleis/momepy/blob/v0.3.0/momepy/utils.py#L117-L156
%time G = generate_graph(graph_notdir_32640, fields)

Wall time: 23.1 s


In [75]:
closest_node = 23361
%time egg = nx.ego_graph(G, closest_node, radius=500, distance='length')

Wall time: 66 ms


In [76]:
%time edglst = nx.to_pandas_edgelist(egg)

Wall time: 3.99 ms


In [77]:
graph_notdir.columns

Index(['link_id', 'from_node', 'to_node', 'geometry'], dtype='object')

In [78]:
edglst.head(2)

,source,target,link_id,length,to_node,geometry,key,from_node
0,22913,22891,60757,98.364760,22913,LINESTRING (46296.88941584923 5918797.88517200...,30378,22891
1,22913,23110,60821,93.087539,23110,LINESTRING (46382.84434481984 5918845.62585193...,30410,22913


In [79]:
len(edglst)

67

In [ ]:
i=0
for i in range(len(gdf_stops)):
    one_stop_gdf = gdf_stops.iloc[[i]].reset_index(drop=True)
    gdf_buff = one_stop_gdf.copy()
    gdf_buff.geometry = gdf_buff.geometry.to_crs('epsg:32640').buffer(1000).to_crs('epsg:4326')
    graph_in_buff = intersect_using_spatial_index(graph_notdir, gdf_buff[['geometry']])
    
    closest_node = find_closest_node(one_stop_gdf, graph_notdir, nodes)
    
    egg = nx.ego_graph(G, closest_node, radius=500, distance='weight')
    edglst = nx.to_pandas_edgelist(egg)
    lst_nodes_end = get_lst_nodes(egg)
    lst_paths_to_ends = get_strt_end_paths(egg, closest_node, lst_nodes_end)
    

In [ ]:
#  для обрезки:
# начало - всегда одно и то же
# найти концы - нет соседей?
# на концах найти следующее ребро, которого нет в эго
# построить кратчайший путь от начала до конца
# из последовательности сделать линию
# обрезать линию по длине

In [121]:
def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    deltas = nodes - node
    dist_2 = np.einsum('ij,ij->i', deltas, deltas)
    return np.argmin(dist_2)

In [123]:
node = closest_node_bui
lst_nodes = (nodes.geometry)
%time clst = closest_node(node, lst_nodes)

ValueError: operands could not be broadcast together with shapes (95434,) (2,) 

In [124]:
nodes2 = np.asarray(lst_nodes)

In [126]:
[node]

In [140]:
nodes2 = nodes.copy()


In [142]:
nodes2['x'] = nodes.geometry.x
nodes2['y'] = nodes.geometry.y


In [143]:
nodes2.head(2)

,nodeID,geometry,x,y
0,1,POINT (49.79937 52.29913),49.799369,52.299129
1,2,POINT (49.90683 52.38109),49.906827,52.381086


In [144]:
nodes2.x, nodes2.y

(0        49.799369
 1        49.906827
 2        49.906918
 3        49.906668
 4        49.906285
            ...    
 95429    49.489766
 95430    49.408140
 95431    49.550115
 95432    49.291967
 95433    50.231702
 Name: x, Length: 95434, dtype: float64,
 0        52.299129
 1        52.381086
 2        52.380958
 3        52.380434
 4        52.379670
            ...    
 95429    53.566953
 95430    53.596254
 95431    53.503658
 95432    53.568337
 95433    53.201384
 Name: y, Length: 95434, dtype: float64)

In [127]:
from scipy.spatial import distance

def closest_node(node, nodes):
    closest_index = distance.cdist([node], nodes).argmin()
    return nodes[closest_index]

In [129]:
a = np.random.randint(1000, size=(50000, 2))

some_pt = (1, 2)



In [131]:
type(nodes2)

numpy.ndarray

In [132]:
type(some_pt)

tuple

In [135]:
node.coords[:]

[(50.1975432, 53.2259105)]

In [137]:
closest_node(nodes2, node.coords[0])

ValueError: XB must be a 2-dimensional array.

In [125]:
nodes2

array([<shapely.geometry.point.Point object at 0x000001D8139381C8>,
       <shapely.geometry.point.Point object at 0x000001D813938B08>, ...,
      dtype=object)